# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [ ]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA
from loguru import logger


In [ ]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

#from IPython.core.pylabtools import figsize


#figsize(12, 8)

#sns.set()

In [ ]:
fifa = pd.read_csv("fifa.csv")

In [ ]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [ ]:
# Sua análise começa aqui.
fifa.head()

In [ ]:
fifa.columns

In [ ]:
fifa.info()

In [ ]:
#existem poucas rows com valor null e decidi exclui-las
fifa_notNull = fifa.dropna()

In [ ]:
fifa.shape # 

In [ ]:
#dropna 48 rows
fifa_notNull.shape 

In [ ]:
#instanciando o PCA
pca = PCA()

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [ ]:
#verificando variancia de componentes
pca.fit_transform(fifa_notNull) 
evr = pca.explained_variance_ratio_
type(evr[0])

In [ ]:
def q1():
    return float(evr[0].round(3))

In [ ]:
g = sns.lineplot(np.arange(len(evr)), np.cumsum(evr))
g.axes.axhline(0.95, ls="--", color="red")
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');

In [ ]:
#número de componentes necessários para explicar 95% da variância total:
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
component_number_var_95 = np.argmax(cumulative_variance_ratio >= 0.95) + 1 
type(component_number_var_95)

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [ ]:
def q2():
    return int(component_number_var_95)

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [ ]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [ ]:
pca_2 = PCA(n_components=2)
pca_2.fit(fifa_notNull) 
answer = np.dot(pca_2.components_,x)
(float(answer[0].round(3)),float(answer[1].round(3)))

In [ ]:
def q3():
    return (float(answer[0].round(3)),float(answer[1].round(3)))

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE


In [ ]:
x = fifa_notNull.drop(columns='Overall')
y = fifa_notNull['Overall']

reg = LinearRegression()
rfe = RFE(reg, 5)
    
est = rfe.fit(x,y)
list(x.columns[est.support_])

In [ ]:
def q4():
   return list(x.columns[est.support_])